<a href="https://colab.research.google.com/github/Ash100/Density_Functional_Theory/blob/main/Vibrational_Frequencies_and_Thermodynamic_Analysis_DFT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

My name is **Dr. Ashfaq Ahmad**, working in the field of structure biology and bioinformatics. The DFT notebook is designed for teaching and research purposes. Commercial usage of this code is prohibited. Detailed video tutorial of this notebook can be found on [Bioinformatic Insights](www.youtube.com/@Bioinformaticsinsights).

This notebook calculates the Vibrational Frequencies and thermodynamic properties. For basic DFT calculation please look in [Here](here).

In [ ]:
#@title Install necessary libraries
!pip install pyscf ase matplotlib numpy ipywidgets pubchempy rdkit reportlab

In [2]:
#@title Import libraries
from pyscf import gto, dft, hessian
import matplotlib.pyplot as plt
import numpy as np

# Explanation: Introduction to Frequency Calculations
"""
# Frequency Calculations in Quantum Chemistry

Frequency calculations are used to determine the vibrational frequencies of a molecule. These frequencies correspond to the normal modes of vibration, which are collective motions of the atoms in the molecule. The frequencies are related to the second derivatives of the energy with respect to the nuclear coordinates (Hessian matrix).

## Key Concepts:
1. **Hessian Matrix**: A matrix of second derivatives of the energy with respect to the nuclear coordinates. It describes the curvature of the potential energy surface.
2. **Normal Modes**: Collective motions of atoms in a molecule that represent the fundamental vibrations.
3. **Vibrational Frequencies**: The frequencies of these normal modes, which can be measured experimentally using techniques like infrared (IR) spectroscopy.

## Steps in Frequency Calculations:
1. **Geometry Optimization**: The molecule is first optimized to its equilibrium geometry (minimum energy configuration).
2. **Hessian Calculation**: The Hessian matrix is computed at the optimized geometry.
3. **Frequency Analysis**: The Hessian matrix is diagonalized to obtain the vibrational frequencies and normal modes.
"""



In [ ]:
#@title Step 1: Define the Molecule - Example: Water (H2O)
mol = gto.Mole()
mol.atom = '''
O  0.000000  0.000000  0.000000
H  0.757000  0.586000  0.000000
H -0.757000  0.586000  0.000000
'''
mol.basis = 'sto-3g'  # Basis set
mol.build()

# Explanation: Molecule Definition
"""
## Molecule Definition
- The molecule is defined using its atomic coordinates.
- The `sto-3g` basis set is used for simplicity. For more accurate results, larger basis sets (e.g., `6-31g*`) can be used.
"""

In [ ]:
#@title Step 2: Perform Geometry Optimization
# Use DFT with B3LYP functional for optimization
mf = dft.RKS(mol)
mf.xc = 'b3lyp'
mf.kernel()

# Explanation: Geometry Optimization
"""
## Geometry Optimization
- The molecule is optimized to its equilibrium geometry using Density Functional Theory (DFT) with the B3LYP functional.
- The optimized geometry corresponds to the minimum energy configuration on the potential energy surface.
"""

In [5]:
#@title Step 3: Calculate the Hessian Matrix
# Compute the Hessian matrix (second derivatives of the energy)
hess = mf.Hessian().kernel()


# Explanation: Hessian Matrix
"""
## Hessian Matrix
- The Hessian matrix is computed at the optimized geometry.
- It describes the curvature of the potential energy surface and is used to determine the vibrational frequencies.
"""

In [6]:
#@title Step 4: Perform Frequency Analysis
# Diagonalize the Hessian matrix to obtain vibrational frequencies
from pyscf.hessian import thermo
freq_info = thermo.harmonic_analysis(mol, hess)

# Extract vibrational frequencies (in cm^-1)
frequencies = freq_info['freq_wavenumber']

# Explanation: Frequency Analysis
"""
## Frequency Analysis
- The Hessian matrix is diagonalized to obtain the vibrational frequencies.
- The frequencies are given in wavenumbers (cm^-1), which are commonly used in spectroscopy.
"""

In [ ]:
#@title Step 5: Visualize the Results
# Print the vibrational frequencies
print("Vibrational Frequencies (cm^-1):")
for i, freq in enumerate(frequencies):
    print(f"Mode {i+1}: {freq:.2f} cm^-1")

# Plot the vibrational frequencies
plt.figure(figsize=(8, 6))
plt.bar(range(1, len(frequencies) + 1), frequencies, color='blue')
plt.xlabel('Vibrational Mode')
plt.ylabel('Frequency (cm^-1)')
plt.title('Vibrational Frequencies of the Molecule')
plt.show()

# Explanation: Visualization
"""
## Visualization of Vibrational Frequencies
- The vibrational frequencies are plotted as a bar chart.
- Each bar represents a normal mode of vibration.
- The height of the bar corresponds to the frequency of the mode.
"""

In [ ]:
#@title Step 6: Thermodynamic Analysis (Optional)
# Calculate thermodynamic properties (e.g., zero-point energy, enthalpy, entropy)
thermo_data = thermo.thermo(mf, freq_info['freq_au'], temperature=298.15, pressure=1.0)

# Print the structure of thermo_data to debug
print("Thermo Data Structure:")
print(thermo_data)

# Extract thermodynamic properties
zpe = thermo_data.get('ZPE', ('N/A', 'N/A'))[0]  # Zero-point energy
enthalpy = thermo_data.get('H_tot', ('N/A', 'N/A'))[0]  # Enthalpy
entropy = thermo_data.get('S_tot', ('N/A', 'N/A'))[0]  # Entropy

# Print thermodynamic properties
print("\nThermodynamic Properties:")
print(f"Zero-Point Energy: {zpe} Hartree")
print(f"Enthalpy: {enthalpy} Hartree")
print(f"Entropy: {entropy} Hartree/K")

# Explanation: Thermodynamic Analysis
"""
## Thermodynamic Analysis
- The vibrational frequencies are used to calculate thermodynamic properties such as:
  - **Zero-Point Energy (ZPE)**: The energy of the molecule at 0 K.
  - **Enthalpy**: The total energy of the molecule, including vibrational contributions.
  - **Entropy**: A measure of the disorder of the system.
- These properties are calculated at a specified temperature (298.15 K) and pressure (1 atm).
"""

In [ ]:
#@title Step 7: Save Results (Optional)
# Save the vibrational frequencies to a file
with open('vibrational_frequencies.txt', 'w') as f:
    f.write("Vibrational Frequencies (cm^-1):\n")
    for i, freq in enumerate(frequencies):
        f.write(f"Mode {i+1}: {freq:.2f} cm^-1\n")

# Explanation: Saving Results
"""
## Saving Results
- The vibrational frequencies are saved to a text file for further analysis.
- This file can be used to compare with experimental data or for additional calculations.
"""

In [ ]:
#@title Testing your own compounds from the PubChem
#!pip install pubchempy rdkit pyscf matplotlib numpy ipywidgets

# Import libraries
import pubchempy as pcp
from rdkit import Chem
from rdkit.Chem import AllChem
from pyscf import gto, dft, hessian
from pyscf.hessian import thermo
import matplotlib.pyplot as plt
import numpy as np
import ipywidgets as widgets
from IPython.display import display

# Create a text box and button for user input
cid_input = widgets.Text(value='', placeholder='Enter PubChem CID (e.g., 2519 for caffeine)', description='CID:')
calculate_button = widgets.Button(description="Generate Frequency Report")

# Output area to display results
output = widgets.Output()

# Function to handle button click
def on_calculate_button_click(b):
    with output:
        output.clear_output()  # Clear previous output
        try:
            cid = int(cid_input.value)
            print(f"Fetching molecule with CID: {cid}")

            # Fetch the molecule and generate 3D coordinates
            atoms, coords = fetch_and_optimize_molecule(cid)

            # Perform DFT calculation and analyze the frequencies
            report, frequencies, thermo_data = calculate_and_analyze_frequencies(atoms, coords)

            # Display the report
            print("\n--- Frequency Analysis Report ---")
            print(report)

            # Plot the vibrational frequencies
            plot_vibrational_frequencies(frequencies)

            # Save the report as a text file
            save_report_to_file(report, frequencies, thermo_data)
            print("\nReport saved as 'frequency_report.txt'.")
        except Exception as e:
            print(f"An error occurred: {e}")

# Attach the function to the button
calculate_button.on_click(on_calculate_button_click)

# Display the input box, button, and output area
display(cid_input, calculate_button, output)

# Function to fetch molecule from PubChem and generate 3D coordinates
def fetch_and_optimize_molecule(cid):
    # Fetch the molecule from PubChem
    compound = pcp.Compound.from_cid(cid)
    smiles = compound.canonical_smiles
    print(f"Molecule Name: {compound.iupac_name}")
    print(f"Molecular Formula: {compound.molecular_formula}")
    print(f"SMILES: {smiles}")

    # Convert SMILES to 3D structure using RDKit
    mol = Chem.MolFromSmiles(smiles)
    mol = Chem.AddHs(mol)  # Add hydrogens
    AllChem.EmbedMolecule(mol)  # Generate 3D coordinates
    AllChem.MMFFOptimizeMolecule(mol)  # Optimize geometry

    # Extract atomic coordinates
    conf = mol.GetConformer()
    atoms = [mol.GetAtomWithIdx(i).GetSymbol() for i in range(mol.GetNumAtoms())]
    coords = [conf.GetAtomPosition(i) for i in range(mol.GetNumAtoms())]

    return atoms, coords

# Function to perform DFT calculation and analyze frequencies
def calculate_and_analyze_frequencies(atoms, coords):
    # Define the molecule in PySCF
    mol_pyscf = gto.Mole()
    mol_pyscf.atom = [[atom, (coord.x, coord.y, coord.z)] for atom, coord in zip(atoms, coords)]
    mol_pyscf.basis = 'sto-3g'  # Basis set
    mol_pyscf.build()

    # Perform DFT calculation
    mf = dft.RKS(mol_pyscf)
    mf.xc = 'b3lyp'  # Use B3LYP functional
    mf.kernel()

    # Calculate the Hessian matrix
    hess = mf.Hessian().kernel()

    # Perform frequency analysis
    freq_info = thermo.harmonic_analysis(mol_pyscf, hess)

    # Extract vibrational frequencies (in cm^-1)
    frequencies = freq_info['freq_wavenumber']

    # Perform thermodynamic analysis
    thermo_data = thermo.thermo(mf, freq_info['freq_au'], temperature=298.15, pressure=1.0)

    # Generate the report
    report = f"""
--- Frequency Analysis Report ---
1. Molecule Information:
   - Name: {pcp.Compound.from_cid(int(cid_input.value)).iupac_name}
   - Molecular Formula: {pcp.Compound.from_cid(int(cid_input.value)).molecular_formula}
   - SMILES: {pcp.Compound.from_cid(int(cid_input.value)).canonical_smiles}

2. Vibrational Frequencies (cm^-1):
"""
    for i, freq in enumerate(frequencies):
        report += f"   - Mode {i+1}: {freq:.2f} cm^-1\n"

    report += f"""
3. Thermodynamic Properties:
   - Zero-Point Energy: {thermo_data.get('ZPE', ('N/A', 'N/A'))[0]} Hartree
   - Enthalpy: {thermo_data.get('H_tot', ('N/A', 'N/A'))[0]} Hartree
   - Entropy: {thermo_data.get('S_tot', ('N/A', 'N/A'))[0]} Hartree/K
"""

    return report, frequencies, thermo_data

# Function to plot the vibrational frequencies
def plot_vibrational_frequencies(frequencies):
    plt.figure(figsize=(8, 6))
    plt.bar(range(1, len(frequencies) + 1), frequencies, color='blue')
    plt.xlabel('Vibrational Mode')
    plt.ylabel('Frequency (cm^-1)')
    plt.title('Vibrational Frequencies of the Molecule')
    plt.show()

# Function to save the report to a file
def save_report_to_file(report, frequencies, thermo_data):
    with open('frequency_report.txt', 'w') as f:
        f.write(report)
        f.write("\nVibrational Frequencies (cm^-1):\n")
        for i, freq in enumerate(frequencies):
            f.write(f"Mode {i+1}: {freq:.2f} cm^-1\n")
        f.write("\nThermodynamic Properties:\n")
        f.write(f"Zero-Point Energy: {thermo_data.get('ZPE', ('N/A', 'N/A'))[0]} Hartree\n")
        f.write(f"Enthalpy: {thermo_data.get('H_tot', ('N/A', 'N/A'))[0]} Hartree\n")
        f.write(f"Entropy: {thermo_data.get('S_tot', ('N/A', 'N/A'))[0]} Hartree/K\n")

# Conclusion
"""
# Conclusion
- Frequency calculations provide valuable insights into the vibrational properties of molecules.
- These calculations are essential for interpreting experimental spectra (e.g., IR spectroscopy) and understanding molecular stability and reactivity.
- By combining geometry optimization, Hessian calculations, and frequency analysis, we can fully characterize the vibrational behavior of a molecule.
"""